In [1]:
import gc
import sys
from statistics import mean
import time
import torch
from tqdm import tqdm
from torchinfo import summary
import numpy as np
import itertools
from pathlib import Path
from scipy import special
import matplotlib.pyplot as plt
import datetime
import torch.linalg as LA
from torch.distributions.laplace import Laplace

import utils
import models
import learning_utils
from configurations import args_parser, arguments



In [3]:
start_time = time.time()
args = arguments()
#boardio is for the the tensorboardx prensation and textio is for written documentation
boardio, textio, best_val_acc, path_best_model, last_model_path = utils.initializations(args)
textio.cprint(str(args) if args.__class__.__name__ == 'Namespace' else str(vars(args)))

#mnist_train_data, mnist_test_loader  = utils.data(args)





{'exp_name': None, 'eval': False, 'data': 'mnist', 'norm_std': 0.5, 'norm_mean': 0.5, 'train_batch_size': 20, 'test_batch_size': 1000, 'model': 'cnn2', 'num_users': 30, 'num_users_per_round': 5, 'local_epochs': 1, 'local_iterations': 100, 'global_epochs': 200, 'tau_min': 0.05, 'privacy_noise': 'laplace', 'epsilon_bar': 400, 'optimizer': 'sgd', 'lr': 0.01, 'momentum': 0.5, 'lr_scheduler': False, 'device': 'cpu', 'seed': 0, 'zeta_coeff': 1.05, 'alpha': 1, 'beta': 2, 'gamma': 0.1, 'max_seconds': 300, 'method_choosing_users': 'BSFL brute', 'data_truncation': 1000, 'choosing_users_verbose': True, 'save_best_model': False, 'privacy': False, 'privacy_choosing_users': True, 'epsilon_sum_deascent_coeff': 0.01}


In [4]:
# data
train_data, test_loader = utils.data(args)
#input in the CNNs is the number of channels and in linear models is the size of the flatten pictures
input, output, train_data, val_loader = utils.data_split(train_data, len(test_loader.dataset), args)

# model
if args.model == 'mlp':
    global_model = models.FC2Layer(input, output)
elif args.model == 'cnn2':
    global_model = models.CNN2Layer(input, output, args.data)
elif args.model == 'cnn3':
    if args.data == 'cifar10':
        global_model = models.CNN3LayerCifar()
    else:
        global_model = models.CNN3LayerMnist()
elif args.model == 'cnn5':
    if args.data == 'mnist' or args.data == 'fashion mnist':
        raise ValueError('CNN5 is not supported for MNIST type datasets')
    global_model = models.CNN5Layer(input, output)
elif args.model == 'linear':
    global_model = models.Linear(input, output)



textio.cprint(str(summary(global_model)).encode('utf-8', errors='ignore').decode('utf-8', errors='ignore'))
global_model.to(args.device)

train_criterion = torch.nn.CrossEntropyLoss(reduction='mean')
test_criterion = torch.nn.CrossEntropyLoss(reduction='sum')

local_models = utils.federated_setup(global_model, train_data, args, i_i_d=True)
utils.update_data_equility_partititon(local_models, args)



Layer (type:depth-idx)                   Param #
CNN2Layer                                --
├─Conv2d: 1-1                            156
├─Conv2d: 1-2                            906
├─Linear: 1-3                            4,850
├─Linear: 1-4                            510
├─Dropout: 1-5                           --
├─Dropout: 1-6                           --
├─BatchNorm2d: 1-7                       12
├─BatchNorm1d: 1-8                       100
Total params: 6,534
Trainable params: 6,534
Non-trainable params: 0


In [5]:


choices_table = np.zeros((args.global_epochs, args.num_users))
num_of_obs_arr = np.zeros((1,args.num_users))
train_loss_list = []
val_acc_list = []
val_losses_list = []
l1_norm_avg_deltha_theta_list = []
global_epochs_time_list = []


time_counter = 0
for global_epoch in tqdm(range(1, args.global_epochs+1)):
    """Part 1: Choosing Users"""
    for usr_idx in range(args.num_users):
        local_models[usr_idx].update_g(global_epoch)
        local_models[usr_idx].update_ucb(global_epoch)
    
    if args.choosing_users_verbose:
        print(f"iteration{global_epoch}")
    a=time.time()
    rounds_choise = utils.choose_users(local_models, args, method=args.method_choosing_users)

    
    #choices_table[global_epoch-1, rounds_choise] = 1
    num_of_obs_arr[0,rounds_choise] += 1
    for usr_idx in rounds_choise:
        local_models[usr_idx].update_emp_avg()
        local_models[usr_idx].update_privacy_violation_and_reward()
        local_models[usr_idx].increase_num_of_obs()
        if args.choosing_users_verbose:
            print(f"user {usr_idx}, g: {local_models[usr_idx].g}, ucb: {local_models[usr_idx].ucb}, num_of_obs: {local_models[usr_idx].num_of_obs}, curr_delay = {local_models[usr_idx].last_access_time}, privacy reward: {local_models[usr_idx].privacy_reward}")
    
    max_delay = max([local_models[i].last_access_time for i in rounds_choise])
    if args.choosing_users_verbose:
        print(f"max_delay = {max_delay:.2f} seconds")
    
    
#     """Part 2: Training"""
#     learning_utils.distribute_model(local_models, global_model)
#     users_avg_loss_over_local_epochs = []

#     for user_idx in rounds_choise:
#         user_loss = []
#         for local_epoch in range(args.local_epochs):
#             user = local_models[user_idx]
#             train_loss = learning_utils.train_one_epoch(user, train_criterion, args)
#             if args.lr_scheduler:
#                 user.scheduler.step(train_loss)
#             user_loss.append(train_loss)
#         users_avg_loss_over_local_epochs.append(mean(user_loss))
    
#     avg_loss_over_chosen_users_curr_global_epoch = mean(users_avg_loss_over_local_epochs)
#     train_loss_list.append(avg_loss_over_chosen_users_curr_global_epoch)


#     avg_deltha_theta = learning_utils.Fed_avg_models(local_models, global_model, rounds_choise, args)
#     # l1_norm_avg_deltha_theta = sum(LA.vector_norm(param.flatten(),1) for param in avg_deltha_theta.values())
#     # l1_norm_avg_deltha_theta_list.append(l1_norm_avg_deltha_theta)
#     # print(f"l1_norm_avg_deltha_theta is {l1_norm_avg_deltha_theta}")
    

#     val_acc, val_loss = learning_utils.test(test_loader, global_model, test_criterion, args)
#     val_acc_list.append(val_acc) ; val_losses_list.append(val_loss)
    

#     # boardio.add_scalars("Losses over time in seconds", {"train_loss":avg_loss_over_chosen_users_curr_global_epoch,
#     #                                     "val_loss": val_loss}, time.time()-start_time)
#     # boardio.add_scalar('Val Accuracy', val_acc, time.time()-start_time)


#     time_counter += max_delay
#     print((f"global epoch {global_epoch} has been done artifficialy in {max_delay:.2f} secs, the total time by now is {time_counter:.2f} \n with avg train loss {avg_loss_over_chosen_users_curr_global_epoch:.3f}, val loss {val_loss:.3f}, avg val acc {val_acc:.2f}%"))
#     global_epochs_time_list.append(time_counter)
#     gc.collect()


#     if val_acc > best_val_acc and args.save_best_model:
#         best_val_acc = val_acc
#         torch.save({"model's state dict":global_model.state_dict(),
#                     "train_loss_list": train_loss_list,
#                     "val_acc_list": val_acc_list,
#                     "val_losses_list": val_losses_list,
#                     "global_epochs_time_list": global_epochs_time_list,
#                     "num_of_users": args.num_users,
#                     "num_of_users_per_round": args.num_users_per_round,
#                     "l1_norm_avg_deltha_theta_list": l1_norm_avg_deltha_theta_list}
#                     , path_best_model)
    
    
#     torch.save({"model's state dict":global_model.state_dict(),
#                 "train_loss_list": train_loss_list,
#                 "val_acc_list": val_acc_list,
#                 "val_losses_list": val_losses_list,
#                 "global_epochs_time_list": global_epochs_time_list,
#                 "num_of_obs_arr": num_of_obs_arr.reshape(-1),
#                 "global_epoch": global_epoch,
#                 "num_of_users": args.num_users,
#                 "num_of_users_per_round": args.num_users_per_round,
#                 "l1_norm_avg_deltha_theta_list": l1_norm_avg_deltha_theta_list}
#                 , last_model_path)

#     if time_counter > args.max_seconds:
#         break

# users_idxs = tuple([str(x) for x in range(1,args.num_users+1)])
# fig, ax = plt.subplots()
# ax.bar(users_idxs, num_of_obs_arr.reshape(-1), width = 0.4)
# ax.set_title("Number of times each user was chosen")
# ax.set_ylabel("number of times")
# ax.set_xlabel("user index")
# #boardio.add_figure("Number of times each user was chosen", fig, global_epoch)
# plt.savefig(last_model_path.parent / "Number of times each user was chosen.png")

# #choices_table = choices_table.cumsum(axis=0)


  0%|          | 0/200 [00:00<?, ?it/s]

iteration1


  0%|          | 1/200 [00:00<01:33,  2.13it/s]

user 5, g: 0.027777777777777776, ucb: inf, num_of_obs: 1, curr_delay = 0.100240881835021, privacy reward: 0.9900498337491681
user 9, g: 0.027777777777777776, ucb: inf, num_of_obs: 1, curr_delay = 0.13641105284531385, privacy reward: 0.9900498337491681
user 10, g: 0.027777777777777776, ucb: inf, num_of_obs: 1, curr_delay = 0.14561784036417957, privacy reward: 0.9900498337491681
user 14, g: 0.027777777777777776, ucb: inf, num_of_obs: 1, curr_delay = 0.19649721564120354, privacy reward: 0.9900498337491681
user 29, g: 0.027777777777777776, ucb: inf, num_of_obs: 1, curr_delay = 0.8842598609443887, privacy reward: 0.9900498337491681
max_delay = 0.88 seconds
iteration2


  1%|          | 2/200 [00:00<01:27,  2.26it/s]

...

user 23, g: 0.0009182736455463729, ucb: 2.4851725722446227, num_of_obs: 28, curr_delay = 0.7982840302355213, privacy reward: 0.7557837414557268
user 24, g: 0.0009182736455463729, ucb: 2.4839666253390114, num_of_obs: 28, curr_delay = 0.8468950431579654, privacy reward: 0.7557837414557268
user 25, g: 0.0009182736455463729, ucb: 2.4826474529197613, num_of_obs: 28, curr_delay = 0.8440454069580183, privacy reward: 0.7557837414557268
user 26, g: 0.0009182736455463729, ucb: 2.4820727136856378, num_of_obs: 28, curr_delay = 0.8471641322659539, privacy reward: 0.7557837414557268
user 28, g: 0.0009182736455463729, ucb: 2.4809299829777585, num_of_obs: 28, curr_delay = 0.8992444074415359, privacy reward: 0.7557837414557268
max_delay = 0.90 seconds
iteration199


100%|█████████▉| 199/200 [01:41<00:00,  1.85it/s]

user 9, g: -1.753603304069201e-05, ucb: 2.497269807622689, num_of_obs: 35, curr_delay = 0.16003510123803016, privacy reward: 0.704688089718715
user 11, g: 7.014413216276711e-07, ucb: 2.4823401186560625, num_of_obs: 34, curr_delay = 0.17965351664300214, privacy reward: 0.7117703227626112
user 14, g: 3.437062475975621e-05, ucb: 2.4822160055177243, num_of_obs: 33, curr_delay = 0.1582149937238337, privacy reward: 0.7189237334319276
user 27, g: 0.0009602731693082938, ucb: 2.482083516773091, num_of_obs: 28, curr_delay = 0.8607888755107017, privacy reward: 0.7557837414557268
user 29, g: 0.0009602731693082938, ucb: 2.480940640204101, num_of_obs: 28, curr_delay = 0.9029108625405724, privacy reward: 0.7557837414557268
max_delay = 0.90 seconds
iteration200


100%|██████████| 200/200 [01:42<00:00,  1.95it/s]

user 0, g: -0.005377777777777778, ucb: 2.652842233357314, num_of_obs: 49, curr_delay = 0.06491582659410691, privacy reward: 0.6126263941844181
user 2, g: -0.0028444444444444454, ucb: 2.496890157694188, num_of_obs: 45, curr_delay = 0.07158690867115235, privacy reward: 0.6376281516217752
user 5, g: -0.0008027777777777787, ucb: 2.4900343115811303, num_of_obs: 40, curr_delay = 0.09604735967851973, privacy reward: 0.6703200460356411
user 6, g: -0.00033611111111111135, ucb: 2.4895221966179197, num_of_obs: 38, curr_delay = 0.1401632423560255, privacy reward: 0.6838614092123576
user 8, g: -6.944444444444442e-05, ucb: 2.487611996689047, num_of_obs: 36, curr_delay = 0.1474409994292305, privacy reward: 0.6976763260710326
max_delay = 0.15 seconds
